In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DefaultDataCollator

In [ ]:
## Load the dataset
df = pd.read_csv('Master/data/synthetic_data/question_abstract_pair.csv')
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
## Load tokenizer to process the question and context field
tokenizer = AutoTokenizer.from_pretrained("RuterNorway/Llama-2-13b-chat-norwegian")

In [ ]:
## Tokenize the training data
tokenized_data_train = tokenizer(train_df['Question'].tolist(), train_df['Abstract'].tolist(), truncation=True, padding=True)
tokenized_data_val = tokenizer(val_df['Question'].tolist(), val_df['Abstract'].tolist(), truncation=True, padding=True)

In [ ]:
## Load the transformers model
model = AutoModelForCausalLM.from_pretrained("RuterNorway/Llama-2-13b-chat-norwegian")

In [ ]:
data_collator = DefaultDataCollator()

In [ ]:
## Load the training_args and the trainer
training_args = TrainingArguments(
    output_dir="Master/trained_models",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data_train,
    eval_dataset=tokenized_data_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)